In [15]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, column_property
from sqlalchemy.ext.hybrid import hybrid_method, hybrid_property
from sqlalchemy.pool import NullPool
import pandas as pd
from datetime import datetime as dt

In [16]:
dashboard_engine = sa.create_engine('sqlite:///dirt_db.sqlite')

In [32]:
switches = {'variable': 'nox_lbs', 'fuels': ['Coal', 'Gas', 'Oil', 'Petcoke'], 'startdate': '2019-01-01T00:00:00', 'enddate': '2019-12-31T00:00:00', 'aggfunc': 'SUM', 'colorvar': 'primary_fuel', 'groupvar': 'country', 'variable_label': 'NOx lbs', 'colorvar_label': 'Primary Fuel'}

switches['startdate'] = dt.strptime(switches['startdate'], '%Y-%m-%dT%H:%M:%S').strftime("%Y-%m-%d")
switches['enddate'] = dt.strptime(switches['enddate'], '%Y-%m-%dT%H:%M:%S').strftime("%Y-%m-%d")

query = f"""
        SELECT
            a.datetime_utc,
            b.{switches['groupvar']},
            {switches['aggfunc']}(a.pred_{switches['variable']}) as {switches['variable']},
            b.{switches['colorvar']} as {switches['colorvar']},
            SUM(b.capacity_mw) as capacity_mw
        FROM predictions a
        LEFT JOIN plant b
            ON a.plant_id_wri = b.plant_id_wri
        WHERE datetime_utc BETWEEN '{str(switches['startdate'])}' AND '{str(switches['enddate'])}'
        GROUP BY b.{switches['groupvar']}, a.datetime_utc
        ORDER BY b.{switches['groupvar']}
        """
        # WHERE timestamp >= '{str(switches['startdate'])}'::date AND timestamp <= '{str(switches['enddate'])}'::date
        # {switches['aggfunc']}(cf),    

df = pd.read_sql(query, dashboard_engine)
df

,datetime_utc,country,nox_lbs,primary_fuel,capacity_mw
0,2019-01-06 00:00:00,AFG,6005.484863,Gas,42.0
1,2019-01-13 00:00:00,AFG,6751.494629,Gas,42.0
2,2019-01-20 00:00:00,AFG,10001.279297,Gas,42.0
3,2019-01-27 00:00:00,AFG,9997.672852,Gas,42.0
4,2019-02-03 00:00:00,AFG,9698.714844,Gas,42.0
...,...,...,...,...,...
7899,2019-12-01 00:00:00,ZWE,77159.843750,Coal,920.0
7900,2019-12-08 00:00:00,ZWE,79823.773438,Coal,920.0
7901,2019-12-15 00:00:00,ZWE,85734.585938,Coal,920.0
7902,2019-12-22 00:00:00,ZWE,59458.570312,Coal,920.0


In [18]:
pd.read_sql_table('plant', dashboard_engine)

,plant_id_wri,cum_load,cum_so2,cum_nox,cum_co2,avg_load,avg_so2,avg_nox,avg_co2,pred_cf,...,url,geolocation_source,wepp_id,year_of_capacity_data,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,generation_gwh_2017,estimated_generation_gwh
0,ARG0000002,6.692037e+05,2.447388e+06,1.319710e+06,1.508923e+09,12869.301716,47065.162861,25379.030461,2.901776e+07,0.118623,...,http://energia3.mecon.gov.ar/contenidos/archiv...,Ministerio de Energía y Minería,1011932,2015.0,NaN,NaN,NaN,NaN,NaN,535.362951
1,ARG0000003,1.297086e+05,8.346550e+05,6.302847e+05,7.869048e+08,2494.396038,16051.057331,12120.859861,1.513279e+07,0.740346,...,http://energia3.mecon.gov.ar/contenidos/archiv...,Ministerio de Energía y Minería,None,2015.0,NaN,NaN,NaN,NaN,NaN,325.859739
2,ARG0000004,1.750119e+05,8.908067e+05,4.155079e+05,9.297042e+08,3365.613039,17130.898330,7990.535591,1.787893e+07,0.850512,...,http://energia3.mecon.gov.ar/contenidos/archiv...,Ministerio de Energía y Minería,1048533,2015.0,NaN,NaN,NaN,NaN,NaN,382.722263
3,ARG0000005,1.940572e+05,6.881345e+05,5.248399e+05,9.196999e+08,3731.869621,13233.356171,10093.075632,1.768654e+07,0.886106,...,http://energia3.mecon.gov.ar/contenidos/archiv...,Ministerio de Energía y Minería,None,2015.0,NaN,NaN,NaN,NaN,NaN,407.324673
4,ARG0000006,2.195676e+05,1.129886e+05,3.045448e+05,1.791226e+09,4222.453097,2172.857956,5856.630719,3.444665e+07,0.737200,...,http://energia3.mecon.gov.ar/contenidos/archiv...,Ministerio de Energía y Minería,1011951,2015.0,NaN,NaN,NaN,NaN,NaN,175.654049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8610,WRI1075859,5.476705e+05,4.663125e+06,1.077559e+06,2.050589e+09,10532.124144,89675.478666,20722.290325,3.943440e+07,0.520995,...,https://endcoal.org/global-coal-plant-tracker/,GCPT,None,NaN,NaN,NaN,NaN,NaN,NaN,526.157646
8611,WRI1075860,2.143855e+06,4.636344e+06,4.362100e+06,5.773435e+09,41227.982685,89160.469351,83886.542142,1.110276e+08,0.407887,...,https://endcoal.org/global-coal-plant-tracker/,GCPT,None,NaN,NaN,NaN,NaN,NaN,NaN,3498.316329
8612,WRI1075861,8.699405e+05,5.045185e+06,2.523697e+06,1.941000e+09,16729.625894,97022.778921,48532.629395,3.732692e+07,0.551713,...,https://endcoal.org/global-coal-plant-tracker/,GCPT,None,NaN,NaN,NaN,NaN,NaN,NaN,1049.494899
8613,WRI1075862,1.626841e+06,6.576544e+06,2.087549e+06,3.537435e+09,31285.413139,126471.996695,40145.179612,6.802760e+07,0.619042,...,https://endcoal.org/global-coal-plant-tracker/,GCPT,None,NaN,NaN,NaN,NaN,NaN,NaN,1749.158165


In [20]:
pd.read_sql_table('predictions', dashboard_engine)

,plant_id_wri,datetime_utc,country,latitude,longitude,primary_fuel,estimated_generation_gwh,wri_capacity_mw,pred_gross_load_mw,pred_so2_lbs,pred_nox_lbs,pred_co2_lbs
0,GEODB0040540,2019-01-06,AFG,34.564,69.113,Gas,NaN,42.0,NaN,1837.365601,6005.484863,49241228.0
1,WRI1023776,2019-01-06,DZA,35.866,6.026,Gas,2152.249819,520.0,39266.042969,1726.391235,5893.828613,37106708.0
2,WRI1023795,2019-01-06,DZA,36.892,7.763,Gas,293.864879,71.0,7064.059594,0.000000,0.000000,2268158.0
3,WRI1023777,2019-01-06,DZA,36.599,3.138,Gas,2317.807497,560.0,48578.000000,0.000000,2572.465576,33298784.0
4,WRI1023792,2019-01-06,DZA,36.591,2.922,Gas,413.894196,100.0,2060.413086,0.000000,6145.056152,16438202.0
...,...,...,...,...,...,...,...,...,...,...,...,...
447975,WRI1022376,2019-12-29,ZMB,-17.353,27.186,Coal,NaN,300.0,NaN,2904.101562,27470.878906,109063128.0
447976,WRI1022390,2019-12-29,ZMB,-12.550,28.233,Oil,24.174528,10.0,581.118469,8050.207031,2955.868164,21576880.0
447977,WRI1022386,2019-12-29,ZMB,-12.967,28.633,Oil,120.872642,50.0,123.753326,11604.954102,4977.986328,26606196.0
447978,WRI1022384,2019-12-29,ZMB,-12.817,28.200,Oil,48.349057,20.0,1162.236938,7332.768066,2400.902344,28459432.0
